In [1]:
import pandas as pd
import numpy as np
import tarfile
import io
import glob
import dask.dataframe as dd

# import xgboost as xgb
# from xgboost import XGBClassifier
from bayes_opt import BayesianOptimization
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events

import lightgbm as lgb
import warnings
warnings.simplefilter("ignore")

import myFunc as mf


tar = tarfile.open('../data/raw/20181120_track_features.tar.gz', 'r:gz')
csv_files = tar.getnames()

tf_df_list = []

for csv_file in [csv_files[2], csv_files[4]]:
    csv_contents = tar.extractfile(csv_file).read()
    tf_df_list.append(pd.read_csv(io.BytesIO(csv_contents), encoding='utf8'))

tf_df = pd.concat(tf_df_list, ignore_index=True)
tf_df.rename(columns={'track_id':'track_id_clean'}, inplace=True)

kmean300_df = pd.read_csv('../data/interim/all_data/mbKMeans300clusters.csv', usecols=['track_id','clus'])
kmean300_df.rename(columns={'track_id':'track_id_clean'}, inplace=True)



In [2]:
import json

list_bayesOpt = glob.glob('../models/SVD/LightGBM_BayesOpt_dec17/for20180918/logs_5tracks_10_dec23_newWeights.json')

opt_best_df = pd.DataFrame()
for jsonFile in list_bayesOpt:
    with open(jsonFile) as f:
        optList = []
        for jsonObj in f:
            optDict = json.loads(jsonObj)
            optList.append(optDict)
        
        opt_df = pd.DataFrame(optList)
        opt_df = pd.concat([opt_df.drop(['params'], axis=1), opt_df['params'].apply(pd.Series)], axis=1)
        opt_best_df = pd.concat([opt_best_df,opt_df.sort_values('target',ascending=False).iloc[0:5]])

opt_best_df.sort_values('target',ascending=False)
# opt_df = pd.DataFrame(optList)
# opt_df = pd.concat([opt_df.drop(['params'], axis=1), opt_df['params'].apply(pd.Series)], axis=1)
# opt_df.sort_values('target',ascending=False)

,target,datetime,bagging_fraction,bagging_freq,batch_size,feature_fraction,learning_rate,min_gain_to_split,nFile,num_iterations,num_leaves
24,0.700877,"{'datetime': '2023-01-07 20:25:42', 'elapsed':...",0.8,2.0,10.0,0.700000,0.200000,0.000000,10.0,869.245354,30.000000
10,0.700366,"{'datetime': '2023-01-07 09:14:36', 'elapsed':...",0.8,2.0,10.0,0.769077,0.185150,0.004609,10.0,854.802310,29.757609
17,0.699597,"{'datetime': '2023-01-07 14:20:54', 'elapsed':...",0.8,2.0,10.0,0.778863,0.146755,0.026235,10.0,867.066414,29.262422
27,0.699146,"{'datetime': '2023-01-07 22:40:55', 'elapsed':...",0.8,2.0,10.0,0.752274,0.196186,0.021056,10.0,858.540451,22.942271
16,0.699098,"{'datetime': '2023-01-07 13:39:17', 'elapsed':...",0.8,2.0,10.0,0.733848,0.115966,0.000000,10.0,863.666425,29.045312


In [3]:
opt_best_df.mean()

target                 0.699817
bagging_fraction       0.800000
bagging_freq           2.000000
batch_size            10.000000
feature_fraction       0.746812
learning_rate          0.168812
min_gain_to_split      0.010380
nFile                 10.000000
num_iterations       862.664191
num_leaves            28.201523
dtype: float64

In [4]:
opt_best_df.median()

target                 0.699597
bagging_fraction       0.800000
bagging_freq           2.000000
batch_size            10.000000
feature_fraction       0.752274
learning_rate          0.185150
min_gain_to_split      0.004609
nFile                 10.000000
num_iterations       863.666425
num_leaves            29.262422
dtype: float64

In [5]:
import random

file_list = []

temp_list = []
for logN in range(10):
    temp_list.append('../data/raw/training_set/log_'+str(logN)+'_20180917_000000000000.csv')

random.Random(23).shuffle(temp_list)
file_list += temp_list
    
    
# temp_list = []
# for logN in range(10):
#     temp_list.append('../data/raw/training_set/log_'+str(logN)+'_20180916_000000000000.csv')
    
# random.Random(23).shuffle(temp_list)
# file_list += temp_list


# temp_list = []
# for logN in range(10):
#     temp_list.append('../data/raw/training_set/log_'+str(logN)+'_20180915_000000000000.csv')
    
# random.Random(23).shuffle(temp_list)
# file_list += temp_list


# temp_list = []
# for logN in range(10):
#     temp_list.append('../data/raw/training_set/log_'+str(logN)+'_20180914_000000000000.csv')
    
# random.Random(23).shuffle(temp_list)
# file_list += temp_list


# temp_list = []
# for logN in range(10):
#     temp_list.append('../data/raw/training_set/log_'+str(logN)+'_20180913_000000000000.csv')
    
# random.Random(23).shuffle(temp_list)
# file_list += temp_list


# temp_list = []
# for logN in range(10):
#     temp_list.append('../data/raw/training_set/log_'+str(logN)+'_20180912_000000000000.csv')
    
# random.Random(23).shuffle(temp_list)
# file_list += temp_list


# temp_list = []
# for logN in range(10):
#     temp_list.append('../data/raw/training_set/log_'+str(logN)+'_20180911_000000000000.csv')
    
# random.Random(23).shuffle(temp_list)
# file_list += temp_list

file_list

['../data/raw/training_set/log_7_20180917_000000000000.csv',
 '../data/raw/training_set/log_8_20180917_000000000000.csv',
 '../data/raw/training_set/log_5_20180917_000000000000.csv',
 '../data/raw/training_set/log_6_20180917_000000000000.csv',
 '../data/raw/training_set/log_3_20180917_000000000000.csv',
 '../data/raw/training_set/log_2_20180917_000000000000.csv',
 '../data/raw/training_set/log_9_20180917_000000000000.csv',
 '../data/raw/training_set/log_0_20180917_000000000000.csv',
 '../data/raw/training_set/log_1_20180917_000000000000.csv',
 '../data/raw/training_set/log_4_20180917_000000000000.csv']

In [6]:
len(file_list)

10

In [7]:
import random
from timeit import default_timer as timer #to see how long the computation will take

nFile = 0
batch_size = 10
while nFile < len(file_list):
    start = timer()
    nFile += batch_size
    df_lookup_list = []
    for file in file_list[(nFile-batch_size):min(nFile, len(file_list))]:
        df_lookup_list.append(mf.prep_dfs(file, tf_df, kmean300_df))

    df_lookup = pd.concat(df_lookup_list)
    df_lookup = df_lookup.merge(tf_df.drop(columns = ['key','time_signature','mode']))
    
#     # check whether the column names match with the previous training set
#     if nFile>batch_size:
#         prev_feature_names = lgb.Booster(model_file='../models/SVD/LightGBM_BayesOpt_dec17/for20180918/boost_alltracks_incrementalTrain_5_dec23.txt').feature_name()
#         if bool(set(prev_feature_names) - set(df_lookup.columns)): # if there are missing columns
#             df_lookup[list(set(prev_feature_names) - set(df_lookup.columns))] = 0 # add the missed columns with 0
            
#         if bool(set(df_lookup.columns)- set(prev_feature_names)): # if there are extra columns
#             extra_cols = list(set(df_lookup.columns)- set(prev_feature_names) - set(['session_id','track_id_clean','skip_2','weight']))
#             df_lookup.drop(columns = extra_cols, inplace = True)
    

    dtrain = lgb.Dataset(df_lookup.drop(columns = ['session_id','track_id_clean','skip_2','weight']).astype('float32'), 
                     label=df_lookup['skip_2'],
                     weight = df_lookup['weight'],
                     free_raw_data=False) # https://lightgbm.readthedocs.io/en/latest/FAQ.html#error-messages-cannot-before-construct-dataset

    
    params = {'num_leaves': 25,
              'learning_rate':0.15,
              'metric': 'binary_error',
              'num_iterations':851,
              'bagging_fraction':0.8,
              'bagging_freq':2,
              'feature_fraction':0.73,
              'min_gain_to_split':0.09,
              'objective': 'binary',
              'force_row_wise': True,
              'num_threads': 5,
              'verbosity': 0,
              'tree_learner': 'voting_parallel'} #https://lightgbm.readthedocs.io/en/latest/Parallel-Learning-Guide.html
    
    if nFile == batch_size:
        bst = lgb.train(params, dtrain)
#     else: # continue training on the previous model
#         bst = lgb.train(params, dtrain, init_model='../models/SVD/LightGBM_BayesOpt_dec17/for20180918/boost_alltracks_incrementalTrain_'+str(int(nFile-batch_size))+'_dec23.txt')
        
    bst.save_model('../models/SVD/LightGBM_BayesOpt_dec17/for20180918/boost_alltracks_dec23_newWeights_oldParams.txt')

    print('Runtime per batch: %0.2fs' % (timer() - start))

Runtime per batch: 2333.10s
